# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [2]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data', header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [5]:
df.describe(include='all') ## 690 observations accross the board

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
count,690,690,690.000000,690,690,690,690,690.000000,690,690,690.00000,690,690,690,690.000000,690
unique,3,350,NaN,4,4,15,10,NaN,2,2,NaN,2,3,171,NaN,2
top,b,?,NaN,u,g,c,v,NaN,t,f,NaN,f,g,00000,NaN,-
freq,468,12,NaN,519,519,137,399,NaN,361,395,NaN,374,625,132,NaN,383
mean,NaN,NaN,4.758725,NaN,NaN,NaN,NaN,2.223406,NaN,NaN,2.40000,NaN,NaN,NaN,1017.385507,NaN
std,NaN,NaN,4.978163,NaN,NaN,NaN,NaN,3.346513,NaN,NaN,4.86294,NaN,NaN,NaN,5210.102598,NaN
min,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.00000,NaN,NaN,NaN,0.000000,NaN
25%,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,0.165000,NaN,NaN,0.00000,NaN,NaN,NaN,0.000000,NaN
50%,NaN,NaN,2.750000,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,0.00000,NaN,NaN,NaN,5.000000,NaN
75%,NaN,NaN,7.207500,NaN,NaN,NaN,NaN,2.625000,NaN,NaN,3.00000,NaN,NaN,NaN,395.500000,NaN


In [16]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64

In [18]:
columns = list(df)
for i in columns:
    print('A',i+1, type(df[i][0]))

A 1 <class 'str'>
A 2 <class 'str'>
A 3 <class 'numpy.float64'>
A 4 <class 'str'>
A 5 <class 'str'>
A 6 <class 'str'>
A 7 <class 'str'>
A 8 <class 'numpy.float64'>
A 9 <class 'str'>
A 10 <class 'str'>
A 11 <class 'numpy.int64'>
A 12 <class 'str'>
A 13 <class 'str'>
A 14 <class 'str'>
A 15 <class 'numpy.int64'>
A 16 <class 'str'>


In [23]:
names = {0:'A1',
        1: 'A2',
        2: 'A3',
        3: 'A4',
        4: 'A5',
        5: 'A6',
        6: 'A7',
        7: 'A8',
        8: 'A9',
        9: 'A10',
        10: 'A11',
        11: 'A12',
        12: 'A13',
        13: 'A14',
        14: 'A15',
        15: 'A16'
        }
df.rename(columns=names, inplace=True)

In [65]:
df['A2'].replace('?', 30.83, inplace=True)
df['A2'] = df['A2'].astype(float)

In [64]:
df['A14'].replace('?', 202, inplace=True)
df['A14'] = df['A14'].astype(float)

In [67]:
df['A11'] = df['A11'].astype(float)
df['A15'] = df['A15'].astype(float)

In [68]:
columns = list(df)  #check to see that values are as expected
for i in columns:
    print(i, type(df[i][0]))

A1 <class 'str'>
A2 <class 'numpy.float64'>
A3 <class 'numpy.float64'>
A4 <class 'str'>
A5 <class 'str'>
A6 <class 'str'>
A7 <class 'str'>
A8 <class 'numpy.float64'>
A9 <class 'str'>
A10 <class 'str'>
A11 <class 'numpy.float64'>
A12 <class 'str'>
A13 <class 'str'>
A14 <class 'numpy.float64'>
A15 <class 'numpy.float64'>
A16 <class 'str'>


In [74]:
df['A16'].replace({'+':1, '-':0}, inplace=True)

TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [76]:
df.replace('?', None, inplace=True)

A1     None
A2     None
A3     None
A4     None
A5     None
A6     None
A7     None
A8     None
A9     None
A10    None
A11    None
A12    None
A13    None
A14    None
A15    None
A16    None
dtype: object

In [77]:
df # df looks to be in good shape

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.250,t,t,1.0,f,g,202.0,0.0,1
1,a,58.67,4.460,u,g,q,h,3.040,t,t,6.0,f,g,43.0,560.0,1
2,a,24.50,0.500,u,g,q,h,1.500,t,f,0.0,f,g,280.0,824.0,1
3,b,27.83,1.540,u,g,w,v,3.750,t,t,5.0,t,g,100.0,3.0,1
4,b,20.17,5.625,u,g,w,v,1.710,t,f,0.0,f,s,120.0,0.0,1
5,b,32.08,4.000,u,g,m,v,2.500,t,f,0.0,t,g,360.0,0.0,1
6,b,33.17,1.040,u,g,r,h,6.500,t,f,0.0,t,g,164.0,31285.0,1
7,a,22.92,11.585,u,g,cc,v,0.040,t,f,0.0,f,g,80.0,1349.0,1
8,b,54.42,0.500,y,p,k,h,3.960,t,f,0.0,f,g,180.0,314.0,1
9,b,42.50,4.915,y,p,w,v,3.165,t,f,0.0,t,g,52.0,1442.0,1


## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [120]:
A3GBA16 = df['A3'].groupby(df['A16'])
A3GBA16.describe()

,count,mean,std,min,25%,50%,75%,max
A16,,,,,,,,
0,383.0,3.839948,4.337662,0.0,0.835,2.21,5.00,26.335
1,307.0,5.904951,5.471485,0.0,1.500,4.46,9.52,28.000


In [117]:
columns = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15']
collection = []
for i in columns:
    instance = df[i].groupby(df['A16'])
    collection.append(instance.describe())

In [118]:
collection

[    count unique top freq
 A16                      
 0     383      3   b  262
 1     307      3   b  206,
      count       mean        std    min     25%    50%    75%    max
 A16                                                                 
 0    383.0  29.834909  10.776656  15.17  22.125  27.67  34.79  80.25
 1    307.0  33.701661  12.769835  13.75  23.210  30.67  41.33  76.75,
      count      mean       std  min    25%   50%   75%     max
 A16                                                           
 0    383.0  3.839948  4.337662  0.0  0.835  2.21  5.00  26.335
 1    307.0  5.904951  5.471485  0.0  1.500  4.46  9.52  28.000,
     count unique top freq
 A16                      
 0     383      3   u  263
 1     307      4   u  256,
     count unique top freq
 A16                      
 0     383      3   g  263
 1     307      4   g  256,
     count unique top freq
 A16                      
 0     383     15   c   75
 1     307     15   c   62,
     count unique top freq

In [122]:
import scipy.stats as stats
A11GBA16 = df['A11'].groupby(df['A16'])
A11GBA16.describe()

,count,mean,std,min,25%,50%,75%,max
A16,,,,,,,,
0,383.0,0.631854,1.900049,0.0,0.0,0.0,0.0,20.0
1,307.0,4.605863,6.320242,0.0,0.0,3.0,7.0,67.0


In [137]:
A11GBA16 = A11GBA16.apply(pd.DataFrame)

ValueError: If using all scalar values, you must pass an index

In [134]:
A11_0 = A11GBA16[0].dropna()
A11_1 = A11GBA16[1].dropna()

In [136]:
stats.ttest_ind(A11_0, A11_1)

Ttest_indResult(statistic=-11.667004222431277, pvalue=7.957718568079967e-29)

In [138]:
A2GBA16 = df['A2'].groupby(df['A16'])
A2GBA16.describe()

,count,mean,std,min,25%,50%,75%,max
A16,,,,,,,,
0,383.0,29.834909,10.776656,15.17,22.125,27.67,34.79,80.25
1,307.0,33.701661,12.769835,13.75,23.210,30.67,41.33,76.75


In [141]:
A2GBA16 = A2GBA16.apply(pd.DataFrame)


ValueError: If using all scalar values, you must pass an index

In [142]:
A2GBA16.describe()

,0,1
count,383.000000,307.000000
mean,29.834909,33.701661
std,10.776656,12.769835
min,15.170000,13.750000
25%,22.125000,23.210000
50%,27.670000,30.670000
75%,34.790000,41.330000
max,80.250000,76.750000


In [143]:
A2_0 = A2GBA16[0].dropna()
A2_1 = A2GBA16[1].dropna()

In [144]:
stats.ttest_ind(A2_0, A2_1)

Ttest_indResult(statistic=-4.312348096591245, pvalue=1.8515930318697948e-05)

In [148]:
# pd.crosstab([df.company, df.experience], df.regiment,  margins=True)
GB16 = df.groupby(df['A16'])
GB16.describe()

A11                                                  A14              \
     count      mean       std  min  25%  50%  75%   max  count        mean   
A16                                                                           
0    383.0  0.631854  1.900049  0.0  0.0  0.0  0.0  20.0  383.0  199.741514   
1    307.0  4.605863  6.320242  0.0  0.0  3.0  7.0  67.0  307.0  165.156352   

      ...      A3             A8                                              \
      ...     75%     max  count      mean       std  min    25%    50%  75%   
A16   ...                                                                      
0     ...    5.00  26.335  383.0  1.257924  2.120481  0.0  0.125  0.415  1.5   
1     ...    9.52  28.000  307.0  3.427899  4.120792  0.0  0.750  2.000  5.0   

             
        max  
A16          
0    13.875  
1    28.500  

[2 rows x 48 columns]

In [154]:
CT_16A4 = pd.crosstab(df['A16'], df['A4'])
CT_16A4

A4,?,l,u,y
A16,,,,
0,2,0,263,118
1,4,2,256,45


In [155]:
CT_16A5 = pd.crosstab(df['A16'], df['A5'])
CT_16A5

A5,?,g,gg,p
A16,,,,
0,2,263,0,118
1,4,256,2,45


In [161]:
CT_16A6 = pd.crosstab(df['A16'], df['A6'])
CT_16A6 = CT_16A6.replace('?', 'NaN') ##???????
CT_16A6

A6,?,aa,c,cc,d,e,ff,i,j,k,m,q,r,w,x
A16,,,,,,,,,,,,,,,
0,5,35,75,12,23,11,46,45,7,37,22,27,1,31,6
1,4,19,62,29,7,14,7,14,3,14,16,51,2,33,32


## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

Both of my ttests gave negative statistic values which can be interpreted differently depending on whether or not the test is two tailed or one.  However both gave p values that would have me reject the null hypothesis, i.e. the means are distinct. 